In [1]:
import csv
import random
import names
import math
import datetime
from sklearn import preprocessing
import statistics

In [2]:
#initialize random seed, default value 42 (=answer to everything).  The random seed will determine all of the numbers in the CSV, feel free to adjust for different numbers.
random.seed(42)

This python code will generate the demo data for PowerBI/Tableau in CSV.

The demo data constains information about the following:
* Team Managers
* Account managers
* Products  
  * Category
  * Sub Category
* Daily Sales
* Marketing Campaigns

#### Gaussian function with listlength, bot and top

In [3]:
class Gauss():
    '''
    Creates a list with random gaussian values (taken from normal distribution), except you can specify bottom and top values.
    If top or bottom value is specified then the gaussian value will be simply filtered if it falls above or below.
    
    class gauss(mu, sigma, listlength, bottom, top)
    
    listlength (int) = how long you want the list to be
    decimals = how many decimals you want each element in the list, default = 2. for 0 decimals method returns an integer. 
    '''
    def __init__(self, mu, sigma, listlength, bot= None, top= None, decimals=2):
        self.mu = mu
        self.sigma = sigma
        self.listlength = listlength
        self.bot = bot
        self.top = top
        self.decimals = decimals
        
        
    def generate(self):
        list = []
        
        while len(list) < abs(self.listlength):
            
            x = None
        
            if self.bot is None and self.top is None:
                list.append(random.gauss(self.mu, self.sigma))
            
            
            if self.bot is not None and self.top is not None:
                while x is None or self.bot > x > self.top:
                    x = random.gauss(self.mu,self.sigma)
                
                    if self.bot <= x <= self.top:
                        list.append(x)
                        
            if self.bot is not None and self.top is None:
                while x is None or self.bot > x:
                    x = random.gauss(self.mu,self.sigma)
                    
                    if self.bot <= x:
                        list.append(x)
                        
            if self.bot is None and self.top is not None:
                while x is None or x > self.top:
                    x = random.gauss(self.mu,self.sigma)
                    
                    if x <= self.top:
                        list.append(x)
                        
        
        list = [round(i, self.decimals) for i in list]
        
        if self.decimals <= 0:
            for i in range(len(list)):
                list[i] = int(list[i])
            
                        
        
        return list

In [4]:
class Standardizer():
    """
    Standardizes a list and returns a tuple with the original list, the standardized list, the mu and sigma.
    
    Please note this class uses a sample standard deviation not a population one.
    
    tuple = list, stdz, mu, sig
    
    stdz : standardized version of the list
    """
    
    def standardise(self, list):
        self.list = list
        mu = statistics.mean(self.list)
        sig = statistics.stdev(self.list)
        stdz = [((i - mu)/sig) for i in self.list]
        
        return (self.list, stdz, mu, sig)
        
        

#### Employees

In [5]:

class Employees():
    """
    Creates a list with sales employees that consists of team managers and account managers. Every Team manager has 10 account managers working for him.
    
    class Employees(amount, epm)
    
    amount: Total amount of employees. Default = 50
    epm: employees per manager. Default 10, rounds down when calculating total amount of managers because most firms are greedy.
    
    contains the following lists: id, fullname, firstname, lastname, amount of employees, roles, team number
    """    
    def __init__(self, amount=50, epm=10):
        self.amount = amount #total amount of employees including managers
        self.epm = epm #employees per team manager
        self.id = []  #employee ID
        self.namelist = [] #Namelist of all employees (Firstname and Lastname)
        self.firstnamelist = [] #employee first name
        self.lastnamelist = [] #employee last name
        self.tman = math.trunc(self.amount/self.epm) # amount of teammanagers
        self.rolelist = [] #list with roles
        self.tnum = [] #list with team numbers
        self.temployed = [] #list with employment time (in years)
        self.learning = [] #list with hours of paid training within firm (in hours)
        self.salary = [] # salary
        self.extraversion = [] #extraversion scores
        self.age = [] #age
    
    def generate(self):
        self.tman = math.trunc(self.amount/self.epm) #amount of team managers
    
        #Create all employee ID's
        for i in range(1, self.amount+1):
            self.id.append(i)
    
        # Create namelist
        for i in range(0, self.amount):
            self.namelist.append(names.get_full_name())
            
        #Seperate namelist into a list with firstnames
        for i in range(0, self.amount):
            self.firstnamelist.append(
                self.namelist[i].split()[0]
            )
            
        #seperate namelist into a list with lastnames
        for i in range(0, self.amount):
            self.lastnamelist.append(
                self.namelist[i].split()[1]
            )
            
        #Create list with role description [Account manager, Team manager]
        for i in range(0, self.tman):
            self.rolelist.append(
                "Team manager"
            )
            
        for i in range(self.tman, self.amount):
            self.rolelist.append(
                "Account manager"
            )
            
        #Create list with team numbers
        while self.amount > len(self.tnum):
            kounter = 0
            for i in range(0, self.tman):
                kounter += 1
                self.tnum.append(kounter)
                
        #Create list with team member age
        self.age = Gauss(30,10,self.amount,25,55,0)
        self.age = self.age.generate()
        
        #Create list with team member time employed at the firm
        #Time employed is dependent on the age. 
        for i in self.age:
            ga = Gauss(i-25, i-20, 1, 0, 15, 0)
            self.temployed.append(int(ga.generate()[0]))
            
        #Create list with total training hours
        #Total training hours is dependent on team member employment age
        #The assumption here is a total 100 training hours in a year with employees starting training since 25 years old.
        for i in self.temployed:
            ga = Gauss((i*100)+10, i*100, 1, 10, None, 0)
            self.learning.append(int(ga.generate()[0]))
            
        
        #list with salary
        self.salary = []
        
        ga = Gauss(70000, 10000, self.tman, None, None, -2)
        self.salary = self. salary + ga.generate()
        
        ga = Gauss(50000, 20000, self.amount-self.tman, None, None, 0)
        self.salary = self.salary + ga.generate()
                
             
        #list with extraversion score
        ga = Gauss(0.6, 0.2, self.amount, 0.35, 1, 3)
        self.extraversion = ga.generate()
        
        
        self.output = zip(self.id, self.namelist, self.firstnamelist, self.lastnamelist, self.rolelist, self.tnum, self.temployed, self.learning, self.salary, self.extraversion, self.age)
        
        self.header = ['id', 'name', 'firstname', 'lastname', 'role', 'team_id', 'time_employed_y', 'learning_h', 'salary', 'extraversion', 'age']
        
        

In [6]:

class Products():
    """
    Creates a sheet with differently named products and categories.
    
    products (list) : list of product names, please insert a list otherwise a default list will be created.
    categories (list) : list of product categories
    assigned (Bool) : whether the categories have been assigned to products. If false the categories will be assigned. If true the list of categories has to be as long as the list with products. Example: products=[1,2,3], categories=[a,b,a], assigned = True means product 1 and 3 is category a, product 2 is category b.
    """
    
    def __init__(self, products=[], categories=[]):
        
        self.products = products  #Product list
        self.categories = categories #Category list
        self.marketing = []  #How much marketing has been budgeted per product in €
        self.id = []
        self.price = []
        self.cost = []
        self.stores = []
        
        
        
    def genprod(self, amountprod=50, prodprefix="Product"):
        
        """
        Creates the products
        
        amountprod : total amount of products to be created
        prodprefix : The products will be numbered, the prefix determines what comes before number
        cost : cost of product
        price : price of product
        """
        self.id = []
        self.products = [] #resets list to empty
        self.amountprod = amountprod
        self.prodprefix = prodprefix
        self.cost = []
        self.price = []
        
        for i in range(1,amountprod+1):
            self.products.append(prodprefix + " " + f"{i}")
        
        self.id = [i for i in range(1, len(self.products)+1)]
        
        ga = Gauss(300, 50, len(self.products), 0, None, 0)
        self.price = ga.generate()
        
        for i in self.price:
            ga = Gauss(200, 50, 1, 0, i, 0)
            self.cost += ga.generate()
            
             
        
        
        
            
            
    def gencat(self, amountcat=5, catprefix="Category"):
        """
        Creates the categories
        
        amountcat : total amount of categories to be created
        catprefix : The categories will be numbered, the prefix determines what comes before number
        """
        self.categories = []   # resets categories list to empty
        self.amountcat = amountcat
        self.catprefix = catprefix
        
        if self.categories == []:
            for i in range(1, amountcat+1):
                self.categories.append(catprefix + " " + f"{i}")
        
        #assigns the categories if not assigned. 
        
        n, r = divmod(len(self.products), len(self.categories))
        self.categories = n * self.categories + self.categories[:r]
        
    def genmarkt(self, totalbudget=1000000):
        """
        Generates marketing data.
        
        marketing : marketing budget per product
        stores : in how many stores you can find the product
        """
        self.totalbudget = totalbudget
        self.marketing = [] #marketing per product
        
        
        ### marketing budget per product
        # default: 20% of products (high budget product = hbp) has 80% of marketing budget (mbh)
        hbp = 0.2
        mbh = 0.8
        # 60% of products (low budget product = lbp) has 20% of marketing budget (mbl)
        lbp = 0.6
        mbl = 0.2
        # rest of products has 0 marketing budget
        nbp = 1 - hbp - lbp
        
        total_hbp = int(round(len(self.products) * hbp, 0))
        total_lbp = int(round(len(self.products) * lbp, 0))
        total_nbp = int(round(len(self.products) * nbp, 0))
        
        mbh_per_prod = [int(round(mbh * totalbudget / total_hbp, 0))]
        mbl_per_prod = [int(round(mbl * totalbudget / total_lbp, 0))]
        
        self.marketing = total_hbp * mbh_per_prod + total_lbp * mbl_per_prod + total_nbp * [0]
        random.shuffle(self.marketing)
        
    def genstores(self, mu=250, sigma=250):
        """
        Generates a list that tells in how many stores this product is shown. This list is not (yet) tied to the lists of locations created under class Sales(). 
        """
        self.stores = [] #in how many stores the product can be found
        
        for i in self.products:
            ga = Gauss(mu, sigma, len(self.products), 0, None, 0)
            self.stores = ga.generate()
            
    def genoutput(self):
        """
        Creates all the output
        """
        
        if self.id == []:
            self.genprod()
        
        if self.categories == []:
            self.gencat()
            
        if self.marketing == []:
            self.genmarkt()
        
        if self.stores == []:
            self.genstores()
        
        
        self.header = ['id', 'productname', 'category', 'price', 'cost', 'marketing_budget', 'amount_stores']
        
        self.output = zip(self.id, self.products, self.categories, self.price, self.cost, self.marketing, self.stores)
        
        
        
    
    

In [7]:
class Sales():
    """
    Generates a list with sales for a product on a date by a person.
    
    The sales are calculated with the following model (all independent variables are standardized): 
    Sales_quantity_person = x1 * age + x2 * employment_time + x3 * hours_study + x4 * extraversion score - x5 * (manager_yes/no) + x6 + random_variability
    
    products with a higher marketing budget will get a proportional sales boost.
    
    some employees will get a randomly assigned sale boost.
    
    also some yearly growth is calculated
    
    """
    
    def __init__(self, Employees_class, Products_class):
        
        # all of the columns that will appear on the sheet:
        self.id = []
        self.date = []
        self.product = []
        self.employee = []
        self.quantity = []
        self.price = []
        self.client = []
        self.locations = []
        
        # columns used for calculations taken from class Employees:
        self.age = [] #age of employee
        self.temployed = [] #employment time
        self.growth = 0 # growth rate
        self.learning = [] #learning time
        self.extraversion = [] #extraversion score
        self.manager = [] #manager yes/no
        self.intercept = [] #intercept of regression
        self.variability = [] #variability of regression
        
        #imported objects
        self.Employees_class = Employees_class
        self.Products_class = Products_class
        
    def genlocations(self, location_amount=100, location_prefix="Location"):
        """
        Creates locations where the products have been sold, locations can also represent stores.
        
        location_amount = amount of locations you want generated it wil always generate 1 location that is representing "others". if input is 100 (default) : it will generate 99 locations and 1 location of type "other". 80% of sales will happen in the 99 specified locations and 20% of the sales will happen under "other".
        
        location_prefix =  default "location", will generate list: [location 1, location 2...]
        """
        
        self.location_amount = location_amount
        self.locations = []
        self.location_prefix = location_prefix
        kounter = 1
        
        for i in range(location_amount-1):
            self.locations += [f"{self.location_prefix}" + " " + f"{kounter}"]
            kounter += 1
            
        self.locations = self.locations + ["Other"]
        
    def gensales(self):
        """
        Creates all of the sales data
        
        the variables are tuples and save the list, standardized list (stdz)(z value), mu and sigma (sig).
        
        The sales are calculated with the following model (all independent variables are standardized):   
        Sales_quantity_person = x1 * age + x2 * employment_time + x3 * hours_study + x4 * extraversion score - x5 * (manager_yes/no) + c + random_variability
        
        """
        # first we initiate the standardizer (which creates standardized value out of list)
        standardise = Standardizer()
        
        # first we create the age tuple
        self.age = standardise.standardise(self.Employees_class.age)
        
        #next we create the employment_time tuple
        self.temployed = standardise.standardise(self.Employees_class.temployed)
        
        #next we create the hours_of_study
        self.learning = standardise.standardise(self.Employees_class.learning)
        
        #next we create the extraversion score
        self.extraversion = standardise.standardise(self.Employees_class.learning)
        
        #next we create the list of managers yes/no (1/0)
        managerlist = self.Employees_class.rolelist
        
        managerlist_d = [] # dummy variable of managerlist
        
        for i in managerlist:
            if i == "Team manager":
                managerlist_d.append(1)
            if i == "Account manager":
                managerlist_d.append(0)
        
        mu = statistics.mean(managerlist_d)
        sig = statistics.stdev(managerlist_d)
        
        self.roles_d = (managerlist, managerlist_d, mu, sig)
        
        
        #Here we fill in all the variables from the linear regression equation
        
        x1 = 20
        x2 = 10
        x3 = 50
        x4 = 50
        x5 = 40
        self.c = 200
        
        R = 0.9     # in this case we mean by R how much of the model is explained by the variables
        
        self.pysale = [] #sales amount per person per year. This is a list of lists, pysale[0] = defined as 2021 and goes untill pysale [9] = 2011. pysale[0][10] would be the total sales in 2021 of person with ID 11  (ID starts from 1, python starts from 0). 
        
        #TODO: Currently the data doesn't calculate what happens when someone isn't employed. It calculates assuming that someone has a fixed age and employment time over all these years. This would consequently need timeseries regression analysis, which for the current purpose isn't needed.
        
        #Creates two pools with different means that will be added to the linear regression to create noise. Each pool has its lowerbound and higherbound and therefore it's own mean because it will be input in a uniform random distribution
        poslowerbound = round((self.c*R)-0.5*(self.c*R))
        poshigherbound = round((self.c*R)+0.5*(self.c*R))
        neglowerbound = round(-(self.c*R)-0.5*(self.c*R))
        neghigherbound = round(-(self.c*R)+0.5*(self.c*R))
        
        for i in range(10):
            psale = []  #psale : person sale, without the year factor, psale is a list where every number represent the total sales of that person in a year.
            
            for i in range(self.Employees_class.amount):
                
                #we add some random numbers with an equal chance of being higher or lower. In this way every data entry will be on average either higher or lower than predicted by the linear regression. for example if c = 200 and R 0.8 , then every data entry will be on average either 40 higher or 40 lower. 
                rand = random.randint(0,1)
                
                if rand == 1:
                    rand = random.randint(poslowerbound, poshigherbound)
                
                else:
                    rand = random.randint(neglowerbound,neghigherbound)
                
                linreg = x1 * self.age[1][i] + x2 * self.temployed[1][i] + x3 * self.learning[1][i] + x4 * self.extraversion[1][i] + x5 * self.roles_d[1][i] + self.c + rand
                
                if linreg < 0:
                    linreg = 0
                
                psale.append(round(linreg))

                
            self.pysale += [psale]
            
        #now we create the yearly records for each person every year (multiple records per person per year)
        
        # self.salestup = date, person, prod, price, cost, amount, location = ()
        
        self.tupoutput = [('date', 'person_id', 'person_name', 'prod_id', 'prod_name', 'amount', 'prod_price', 'prod_cost', 'location')] # list containing all tuples
        
        for i1 in range(len(self.pysale)):
            
            year = 2021 - i1
            
            for i2 in range(len(self.pysale[i1])):
                
                total_yearly_amount = self.pysale[i1][i2]   #This is the total yearly amount the employee sold
                
                # we want to break this total yearly amount into ~20 smaller pieces. Each piece will be sold in a random month
                
                small_amount = round (total_yearly_amount / random.randint(15,25))
                
                if small_amount != 0:
                    
                    a, b = divmod(total_yearly_amount, small_amount)
                
                else:
                    a, b = (0,0)
                
                if a > 0:
                    
                    for i3 in range(a):
                        
                        date = datetime.date(year,random.randint(1,12),random.randint(1,28)).strftime('%x')
                        person_id = self.Employees_class.id[i2]
                        person_name = self.Employees_class.namelist[i2]
                        
                        rand = random.randint(0,self.Products_class.amountprod - 1) #chooses a random product from the products list
                        
                        #TODO: marketing budget needs to be tied here, a products with a higher marketing budget must have a higher chance of being chosen.
                        
                        prod_id = self.Products_class.id[rand]
                        prod_name = self.Products_class.products[rand]
                        prod_price = self.Products_class.price[rand]
                        prod_cost = self.Products_class.cost[rand]
                        
                        location = self.locations[random.randint(0,len(self.locations)-1)]
                        
                        self.tupoutput += [(date, person_id, person_name, prod_id, prod_name, small_amount, prod_price, prod_cost, location)]
                        
                if b > 0:
                    
                    date = datetime.date(year,random.randint(1,12),random.randint(1,28)).strftime('%x')
                    person_id = self.Employees_class.id[i2]
                    person_name = self.Employees_class.namelist[i2]
                     
                    rand = random.randint(0,self.Products_class.amountprod - 1) #chooses a randomproduct from the products list
                        
                    #TODO: marketing budget needs to be tied here, a products with a highermarketing budget must have a higher chance of being chosen.
                        
                    prod_id = self.Products_class.id[rand]
                    prod_name = self.Products_class.products[rand]
                    prod_price = self.Products_class.price[rand]
                    prod_cost = self.Products_class.cost[rand]
                     
                    location = self.locations[random.randint(0,len(self.locations)-1)]
                     
                    self.tupoutput += [(date, person_id, person_name, prod_id, prod_name, b, prod_price, prod_cost, location)]
                    
        

In [8]:
class CSVgen():
    
    def __init__(self, Employees_class, Products_class, Sales_class):
        self.Employees_class = Employees_class
        self.Products_class = Products_class
        self.Sales_class = Sales_class
        
    
    def generate(self):
        
        with open('employees.csv', 'w', newline='') as file:
            
            writer = csv.writer(file, dialect='excel')
            
            a = self.Employees_class.header
            b = self.Employees_class.output
            writer.writerow(a)
            writer.writerows(b)
            
        with open('products.csv', 'w', newline='') as file:
            
            writer = csv.writer(file, dialect='excel')
            
            a = self.Products_class.header
            b = self.Products_class.output
            
            
            writer.writerow(a)
            writer.writerows(b)
            
        with open('sales.csv', 'w', newline='') as file:
        
            writer = csv.writer(file, dialect='excel')
            
            a = self.Sales_class.tupoutput
            
            writer.writerows(a)
            
        
            
            


In [9]:
def create():
    employees_object = Employees()
    employees_object.generate()

    Products_object = Products()
    Products_object.genoutput()

    sales = Sales(employees_object, Products_object)
    sales.genlocations()
    sales.gensales()

    write = CSVgen(employees_object, Products_object, sales)
    write.generate()
    
create()

